Code implementation of - **A. Parnak, Y. Baleghi, and J. Kazemitabar, “A novel forgery detection algorithm based on mantissa distribution in digital images,” in 2020 6th Iranian Conference on Signal Processing and Intelligent Systems(ICSPIS), 2020, pp. 1–4.**

In [47]:
import cv2
import pandas as pd
import numpy as np
import os
from math import ceil, log10
from skimage.util import view_as_windows
from scipy.fftpack import dct
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [18]:
def get_patches(image_mat):
    """
    Extract patches rom an image
    :param image_mat: The image as a matrix
    :param stride: The stride of the patch extraction process
    :returns: The patches
    """
    stride=8 #stride is same as window's breadth so that it gives non-overlapping blocks.
    window_shape = (8, 8)
    image_mat=np.array(image_mat)
    
    windows = view_as_windows(image_mat, window_shape, step=stride)
#     print('windows shape:',windows.shape)

    patches = []
    for m in range(windows.shape[0]):
        for n in range(windows.shape[1]):
#             print("window shape: ",windows[m][n].shape)
            patches += [windows[m][n]]
    return patches

In [19]:
def get_zigzag_dct(dct_block):
    matrix=dct_block
    rows=8
    columns=8

    solution=[[] for i in range(rows+columns-1)]

    for i in range(rows):
        for j in range(columns):
            sum=i+j
            if(sum%2 ==0):
                #add at beginning
                solution[sum].insert(0,matrix[i][j])
            else:
                #add at end of the list
                solution[sum].append(matrix[i][j])

    zigzag=[]
    for i in solution:
        for j in i:
            zigzag.append(j)
            
    return(zigzag)

In [20]:
def get_mantissa(f):
    power = int(ceil(log10(abs(f)))) if f != 0 else 0
    mantissa_part = f/(10**power)
    return(mantissa_part)

In [21]:
def mantissa_dct_features(channel_blocks):
    dct_stack=[]
    
    for block in channel_blocks:
        dct_block = dct(block, type=2, n=None, axis=-1, norm=None, overwrite_x=False)
        dct_block_zigzag = get_zigzag_dct(dct_block)
        dct_stack.append(dct_block_zigzag)
        
    dct_stack = np.asarray(dct_stack) #1536X64
    dct_stack = dct_stack.T # 64X1536
#     print("dct_stack shape:",dct_stack.shape)
#     print("dct_stack is:",dct_stack)
    
    dct_stack_25 = dct_stack[:25,:] # Only first 25 rows.
#     print("dct_stack_25 shape:",dct_stack_25.shape)
    rows = dct_stack_25.shape[0]
    cols = dct_stack_25.shape[1]
    
    #Now finding mantissa of every value in dct_stack_25
    for i in range(rows):
        for j in range(cols):
            dct_stack_25[i, j] = get_mantissa(dct_stack_25[i, j])
#     print("dct_stack_25 Mantissa is: ",dct_stack_25)
        
    mantissa_hist_matrix = np.empty([rows,10]) #10 bcoz histogram will have 10 bins.
    for i in range(rows):
        row_modes = dct_stack_25[i,:]
        hist,bins = np.histogram(row_modes,bins = [-1.0,-0.8,-0.6,-0.4,-0.2,0,0.2,0.4,0.6,0.8,1.0]) #10 bins
        mantissa_hist_matrix[i,:] = hist
    
#     print("mantissa_hist_matrix is: ",mantissa_hist_matrix)
#     print("mantissa_hist_matrix shape: ",mantissa_hist_matrix.shape)
    return(mantissa_hist_matrix.flatten()) # 25x10 size


In [22]:
def feature_sub_channel(sub_channel):
    sub_channel_blocks = get_patches(sub_channel) #Gives the 8x8 non-overlapping patches/blocks of sub_channel.
    features = mantissa_dct_features(sub_channel_blocks)
    return(features)

In [30]:
#main function to extract the features.
def feature_extraction(path_to_folder, class_label):
    data_list=[]
    for file_name in os.listdir(path_to_folder):
        path_to_img = os.path.join(path_to_folder,file_name)
        img = cv2.imread(path_to_img)
        
        if np.shape(img) == ():
            continue
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb) #changing to YCrCb color space.
        img_y = img[:,:,0] # the Y channel only.
        img_cb = img[:,:,2] # the Cb channel only.
        
        Fy = feature_sub_channel(img_y)
        Fcb = feature_sub_channel(img_cb)
        
        final_feature = np.concatenate([Fy,Fcb])
#         print("final_feature shape: ",final_feature.shape)
#         print("final_feature is: ",final_feature)
        
        final_feature=list(final_feature)
        final_feature.insert(0,file_name)
        final_feature.insert(1,class_label)
        data_list.append(final_feature)
        
       
    return(data_list)

In [31]:
#CASIA V1.0 dataset
au_path = "YOUR_PATH/CASIA 1.0 dataset/Au"
tp_path1 = "YOUR_PATH/CASIA 1.0 dataset/Modified Tp/Tp/CM"
tp_path2 = "YOUR_PATH/CASIA 1.0 dataset/Modified Tp/Tp/Sp"

data_list1 = feature_extraction(au_path, 0)
data_list2 = feature_extraction(tp_path1, 1)
data_list3 = feature_extraction(tp_path2, 1)

df = pd.DataFrame(data_list1)
df = df.append(pd.DataFrame(data_list2), ignore_index=True)
df = df.append(pd.DataFrame(data_list3), ignore_index=True)


df.rename(columns = {0: "image_names", 1: "label"}, inplace = True)

scaler_norm = MinMaxScaler() 
df.iloc[:,2:] = scaler_norm.fit_transform(df.iloc[:,2:].to_numpy()) # Normalising the values in dataframe.

In [32]:
df.head()

,image_names,label,2,3,4,5,6,7,8,9,...,492,493,494,495,496,497,498,499,500,501
0,Au_txt_0026.jpg,0,0.0,0.0,0.0,0.0,0.0,0.226937,0.105982,0.400000,...,0.273585,0.377953,0.310160,0.387302,0.636066,0.221800,0.420382,0.247475,0.194969,0.293578
1,Au_sec_0080.jpg,0,0.0,0.0,0.0,0.0,0.0,0.367774,0.278283,0.223030,...,0.433962,0.377953,0.529412,0.504762,0.573770,0.141664,0.576433,0.484848,0.339623,0.440367
2,Au_nat_0099.jpg,0,0.0,0.0,0.0,0.0,0.0,0.584256,0.342653,0.000000,...,0.066038,0.141732,0.106952,0.101587,0.331148,0.398785,0.095541,0.005051,0.044025,0.119266
3,Au_art_0059.jpg,0,0.0,0.0,0.0,0.0,0.0,0.431734,0.355657,0.044848,...,0.566038,0.614173,0.582888,0.574603,0.829508,0.066844,0.649682,0.449495,0.383648,0.623853
4,Au_cha_0076.jpg,0,0.0,0.0,0.0,0.0,0.0,0.236777,0.596879,0.078788,...,0.405660,0.511811,0.689840,0.701587,0.695082,0.060008,0.681529,0.686869,0.433962,0.293578


In [33]:
df.shape

(1721, 502)

In [34]:
output_name='CASIA1_baseline_features.csv'
df.to_csv(output_name, index=False)

## Classification using extracted features

In [35]:
array=df.values
x_feature=array[:,2:]
y_label=array[:,1].astype('int')
print(x_feature.shape)
print(y_label.shape)

(1721, 500)
(1721,)


In [41]:
X_train,X_test,Y_train,Y_test=train_test_split(x_feature,y_label,test_size=0.30,random_state=7)

In [37]:
# general check
model_SVC=SVC(kernel='poly', degree=2, C=100, gamma=0.001)

kfold=KFold(n_splits=10, shuffle=True)
cv_results=cross_val_score(model_SVC,X_train,Y_train,cv=kfold,scoring='accuracy')
msg="%s %f (%f)" % ('Training Accuracy: ',cv_results.mean(),cv_results.std())
print(msg)

Training Accuracy:  0.843759 (0.029631)


In [42]:
# SVM hypertuning using GridSeachCV
model_SVC=SVC()

kfold=KFold(n_splits=10)
param_grid = {'C': [1, 10, 100, 500, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['poly'],
             'degree': [2]} #quadratic

grid=GridSearchCV(estimator=model_SVC,param_grid=param_grid,scoring='accuracy',cv=kfold,verbose=3)
grid_result=grid.fit(X_train,Y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV 1/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.926 total time=   0.2s
[CV 2/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.967 total time=   0.2s
[CV 3/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.950 total time=   0.2s
[CV 4/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.926 total time=   0.2s
[CV 5/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.958 total time=   0.2s
[CV 6/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.958 total time=   0.2s
[CV 7/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.958 total time=   0.3s
[CV 8/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.933 total time=   0.2s
[CV 9/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.958 total time=   0.3s
[CV 10/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.925 total time=   0.2s
[CV 1/10] END C=1, degree=2, gamma=0.1, kernel=poly;, score=0.950 total time=   0.2s
[CV 2/10] END C=1, de

[CV 6/10] END C=10, degree=2, gamma=0.0001, kernel=poly;, score=0.600 total time=   0.4s
[CV 7/10] END C=10, degree=2, gamma=0.0001, kernel=poly;, score=0.533 total time=   0.4s
[CV 8/10] END C=10, degree=2, gamma=0.0001, kernel=poly;, score=0.608 total time=   0.4s
[CV 9/10] END C=10, degree=2, gamma=0.0001, kernel=poly;, score=0.483 total time=   0.4s
[CV 10/10] END C=10, degree=2, gamma=0.0001, kernel=poly;, score=0.542 total time=   0.4s
[CV 1/10] END C=100, degree=2, gamma=1, kernel=poly;, score=0.926 total time=   0.2s
[CV 2/10] END C=100, degree=2, gamma=1, kernel=poly;, score=0.967 total time=   0.2s
[CV 3/10] END C=100, degree=2, gamma=1, kernel=poly;, score=0.950 total time=   0.2s
[CV 4/10] END C=100, degree=2, gamma=1, kernel=poly;, score=0.926 total time=   0.2s
[CV 5/10] END C=100, degree=2, gamma=1, kernel=poly;, score=0.958 total time=   0.2s
[CV 6/10] END C=100, degree=2, gamma=1, kernel=poly;, score=0.958 total time=   0.2s
[CV 7/10] END C=100, degree=2, gamma=1, kern

[CV 10/10] END C=500, degree=2, gamma=0.001, kernel=poly;, score=0.892 total time=   0.3s
[CV 1/10] END C=500, degree=2, gamma=0.0001, kernel=poly;, score=0.579 total time=   0.4s
[CV 2/10] END C=500, degree=2, gamma=0.0001, kernel=poly;, score=0.545 total time=   0.4s
[CV 3/10] END C=500, degree=2, gamma=0.0001, kernel=poly;, score=0.587 total time=   0.5s
[CV 4/10] END C=500, degree=2, gamma=0.0001, kernel=poly;, score=0.512 total time=   0.6s
[CV 5/10] END C=500, degree=2, gamma=0.0001, kernel=poly;, score=0.567 total time=   0.6s
[CV 6/10] END C=500, degree=2, gamma=0.0001, kernel=poly;, score=0.600 total time=   0.4s
[CV 7/10] END C=500, degree=2, gamma=0.0001, kernel=poly;, score=0.533 total time=   0.5s
[CV 8/10] END C=500, degree=2, gamma=0.0001, kernel=poly;, score=0.608 total time=   0.6s
[CV 9/10] END C=500, degree=2, gamma=0.0001, kernel=poly;, score=0.483 total time=   0.4s
[CV 10/10] END C=500, degree=2, gamma=0.0001, kernel=poly;, score=0.542 total time=   0.5s
[CV 1/10]

In [49]:
# SVM hypertuning using GridSeachCV 
model_SVC=SVC()

kfold=KFold(n_splits=10)
param_grid = {'C': [1, 10, 100, 500, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['poly','linear'],
             'degree': [2,3]} 

grid=GridSearchCV(estimator=model_SVC,param_grid=param_grid,scoring='accuracy',cv=kfold,verbose=3)
grid_result=grid.fit(X_train,Y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 1/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.926 total time=   0.2s
[CV 2/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.967 total time=   0.2s
[CV 3/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.950 total time=   0.2s
[CV 4/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.926 total time=   0.2s
[CV 5/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.958 total time=   0.2s
[CV 6/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.958 total time=   0.2s
[CV 7/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.958 total time=   0.2s
[CV 8/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.933 total time=   0.2s
[CV 9/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.958 total time=   0.3s
[CV 10/10] END C=1, degree=2, gamma=1, kernel=poly;, score=0.925 total time=   0.3s
[CV 1/10] END C=1, degree=2, gamma=1, kernel=linear;, score=0.901 total time=   0.2s
[CV 2/10] END C=1, 

[CV 6/10] END C=1, degree=2, gamma=0.0001, kernel=linear;, score=0.933 total time=   0.2s
[CV 7/10] END C=1, degree=2, gamma=0.0001, kernel=linear;, score=0.967 total time=   0.2s
[CV 8/10] END C=1, degree=2, gamma=0.0001, kernel=linear;, score=0.942 total time=   0.2s
[CV 9/10] END C=1, degree=2, gamma=0.0001, kernel=linear;, score=0.942 total time=   0.2s
[CV 10/10] END C=1, degree=2, gamma=0.0001, kernel=linear;, score=0.933 total time=   0.2s
[CV 1/10] END C=1, degree=3, gamma=1, kernel=poly;, score=0.917 total time=   0.2s
[CV 2/10] END C=1, degree=3, gamma=1, kernel=poly;, score=0.975 total time=   0.2s
[CV 3/10] END C=1, degree=3, gamma=1, kernel=poly;, score=0.934 total time=   0.2s
[CV 4/10] END C=1, degree=3, gamma=1, kernel=poly;, score=0.926 total time=   0.2s
[CV 5/10] END C=1, degree=3, gamma=1, kernel=poly;, score=0.967 total time=   0.2s
[CV 6/10] END C=1, degree=3, gamma=1, kernel=poly;, score=0.958 total time=   0.2s
[CV 7/10] END C=1, degree=3, gamma=1, kernel=poly;,

[CV 2/10] END C=1, degree=3, gamma=0.0001, kernel=linear;, score=0.950 total time=   0.2s
[CV 3/10] END C=1, degree=3, gamma=0.0001, kernel=linear;, score=0.950 total time=   0.2s
[CV 4/10] END C=1, degree=3, gamma=0.0001, kernel=linear;, score=0.934 total time=   0.2s
[CV 5/10] END C=1, degree=3, gamma=0.0001, kernel=linear;, score=0.925 total time=   0.2s
[CV 6/10] END C=1, degree=3, gamma=0.0001, kernel=linear;, score=0.933 total time=   0.2s
[CV 7/10] END C=1, degree=3, gamma=0.0001, kernel=linear;, score=0.967 total time=   0.2s
[CV 8/10] END C=1, degree=3, gamma=0.0001, kernel=linear;, score=0.942 total time=   0.2s
[CV 9/10] END C=1, degree=3, gamma=0.0001, kernel=linear;, score=0.942 total time=   0.2s
[CV 10/10] END C=1, degree=3, gamma=0.0001, kernel=linear;, score=0.933 total time=   0.2s
[CV 1/10] END C=10, degree=2, gamma=1, kernel=poly;, score=0.926 total time=   0.2s
[CV 2/10] END C=10, degree=2, gamma=1, kernel=poly;, score=0.967 total time=   0.2s
[CV 3/10] END C=10, d

[CV 6/10] END C=10, degree=2, gamma=0.0001, kernel=poly;, score=0.600 total time=   0.4s
[CV 7/10] END C=10, degree=2, gamma=0.0001, kernel=poly;, score=0.533 total time=   0.4s
[CV 8/10] END C=10, degree=2, gamma=0.0001, kernel=poly;, score=0.608 total time=   0.4s
[CV 9/10] END C=10, degree=2, gamma=0.0001, kernel=poly;, score=0.483 total time=   0.4s
[CV 10/10] END C=10, degree=2, gamma=0.0001, kernel=poly;, score=0.542 total time=   0.4s
[CV 1/10] END C=10, degree=2, gamma=0.0001, kernel=linear;, score=0.901 total time=   0.2s
[CV 2/10] END C=10, degree=2, gamma=0.0001, kernel=linear;, score=0.967 total time=   0.2s
[CV 3/10] END C=10, degree=2, gamma=0.0001, kernel=linear;, score=0.959 total time=   0.2s
[CV 4/10] END C=10, degree=2, gamma=0.0001, kernel=linear;, score=0.917 total time=   0.2s
[CV 5/10] END C=10, degree=2, gamma=0.0001, kernel=linear;, score=0.967 total time=   0.2s
[CV 6/10] END C=10, degree=2, gamma=0.0001, kernel=linear;, score=0.950 total time=   0.2s
[CV 7/10

[CV 10/10] END C=10, degree=3, gamma=0.001, kernel=linear;, score=0.917 total time=   0.2s
[CV 1/10] END C=10, degree=3, gamma=0.0001, kernel=poly;, score=0.579 total time=   0.4s
[CV 2/10] END C=10, degree=3, gamma=0.0001, kernel=poly;, score=0.545 total time=   0.4s
[CV 3/10] END C=10, degree=3, gamma=0.0001, kernel=poly;, score=0.587 total time=   0.4s
[CV 4/10] END C=10, degree=3, gamma=0.0001, kernel=poly;, score=0.512 total time=   0.4s
[CV 5/10] END C=10, degree=3, gamma=0.0001, kernel=poly;, score=0.567 total time=   0.4s
[CV 6/10] END C=10, degree=3, gamma=0.0001, kernel=poly;, score=0.600 total time=   0.4s
[CV 7/10] END C=10, degree=3, gamma=0.0001, kernel=poly;, score=0.533 total time=   0.4s
[CV 8/10] END C=10, degree=3, gamma=0.0001, kernel=poly;, score=0.608 total time=   0.4s
[CV 9/10] END C=10, degree=3, gamma=0.0001, kernel=poly;, score=0.483 total time=   0.4s
[CV 10/10] END C=10, degree=3, gamma=0.0001, kernel=poly;, score=0.542 total time=   0.4s
[CV 1/10] END C=10

...

[CV 7/10] END C=100, degree=3, gamma=0.001, kernel=poly;, score=0.533 total time=   0.4s
[CV 8/10] END C=100, degree=3, gamma=0.001, kernel=poly;, score=0.608 total time=   0.4s
[CV 9/10] END C=100, degree=3, gamma=0.001, kernel=poly;, score=0.483 total time=   0.4s
[CV 10/10] END C=100, degree=3, gamma=0.001, kernel=poly;, score=0.542 total time=   0.4s
[CV 1/10] END C=100, degree=3, gamma=0.001, kernel=linear;, score=0.876 total time=   0.2s
[CV 2/10] END C=100, degree=3, gamma=0.001, kernel=linear;, score=0.959 total time=   0.2s
[CV 3/10] END C=100, degree=3, gamma=0.001, kernel=linear;, score=0.950 total time=   0.2s
[CV 4/10] END C=100, degree=3, gamma=0.001, kernel=linear;, score=0.901 total time=   0.2s
[CV 5/10] END C=100, degree=3, gamma=0.001, kernel=linear;, score=0.925 total time=   0.2s
[CV 6/10] END C=100, degree=3, gamma=0.001, kernel=linear;, score=0.950 total time=   0.2s
[CV 7/10] END C=100, degree=3, gamma=0.001, kernel=linear;, score=0.958 total time=   0.2s
[CV 8/

[CV 1/10] END C=500, degree=2, gamma=0.001, kernel=poly;, score=0.884 total time=   0.3s
[CV 2/10] END C=500, degree=2, gamma=0.001, kernel=poly;, score=0.909 total time=   0.3s
[CV 3/10] END C=500, degree=2, gamma=0.001, kernel=poly;, score=0.926 total time=   0.3s
[CV 4/10] END C=500, degree=2, gamma=0.001, kernel=poly;, score=0.893 total time=   0.3s
[CV 5/10] END C=500, degree=2, gamma=0.001, kernel=poly;, score=0.917 total time=   0.3s
[CV 6/10] END C=500, degree=2, gamma=0.001, kernel=poly;, score=0.917 total time=   0.3s
[CV 7/10] END C=500, degree=2, gamma=0.001, kernel=poly;, score=0.975 total time=   0.3s
[CV 8/10] END C=500, degree=2, gamma=0.001, kernel=poly;, score=0.900 total time=   0.3s
[CV 9/10] END C=500, degree=2, gamma=0.001, kernel=poly;, score=0.917 total time=   0.3s
[CV 10/10] END C=500, degree=2, gamma=0.001, kernel=poly;, score=0.892 total time=   0.3s
[CV 1/10] END C=500, degree=2, gamma=0.001, kernel=linear;, score=0.876 total time=   0.2s
[CV 2/10] END C=50

[CV 5/10] END C=500, degree=3, gamma=0.01, kernel=linear;, score=0.925 total time=   0.2s
[CV 6/10] END C=500, degree=3, gamma=0.01, kernel=linear;, score=0.950 total time=   0.2s
[CV 7/10] END C=500, degree=3, gamma=0.01, kernel=linear;, score=0.958 total time=   0.2s
[CV 8/10] END C=500, degree=3, gamma=0.01, kernel=linear;, score=0.917 total time=   0.2s
[CV 9/10] END C=500, degree=3, gamma=0.01, kernel=linear;, score=0.950 total time=   0.2s
[CV 10/10] END C=500, degree=3, gamma=0.01, kernel=linear;, score=0.925 total time=   0.2s
[CV 1/10] END C=500, degree=3, gamma=0.001, kernel=poly;, score=0.694 total time=   0.4s
[CV 2/10] END C=500, degree=3, gamma=0.001, kernel=poly;, score=0.711 total time=   0.4s
[CV 3/10] END C=500, degree=3, gamma=0.001, kernel=poly;, score=0.744 total time=   0.4s
[CV 4/10] END C=500, degree=3, gamma=0.001, kernel=poly;, score=0.694 total time=   0.4s
[CV 5/10] END C=500, degree=3, gamma=0.001, kernel=poly;, score=0.742 total time=   0.4s
[CV 6/10] END 

[CV 8/10] END C=1000, degree=2, gamma=0.01, kernel=poly;, score=0.950 total time=   0.2s
[CV 9/10] END C=1000, degree=2, gamma=0.01, kernel=poly;, score=0.967 total time=   0.2s
[CV 10/10] END C=1000, degree=2, gamma=0.01, kernel=poly;, score=0.933 total time=   0.2s
[CV 1/10] END C=1000, degree=2, gamma=0.01, kernel=linear;, score=0.876 total time=   0.2s
[CV 2/10] END C=1000, degree=2, gamma=0.01, kernel=linear;, score=0.959 total time=   0.2s
[CV 3/10] END C=1000, degree=2, gamma=0.01, kernel=linear;, score=0.950 total time=   0.2s
[CV 4/10] END C=1000, degree=2, gamma=0.01, kernel=linear;, score=0.901 total time=   0.2s
[CV 5/10] END C=1000, degree=2, gamma=0.01, kernel=linear;, score=0.925 total time=   0.2s
[CV 6/10] END C=1000, degree=2, gamma=0.01, kernel=linear;, score=0.950 total time=   0.2s
[CV 7/10] END C=1000, degree=2, gamma=0.01, kernel=linear;, score=0.958 total time=   0.2s
[CV 8/10] END C=1000, degree=2, gamma=0.01, kernel=linear;, score=0.917 total time=   0.2s
[CV 

[CV 1/10] END C=1000, degree=3, gamma=0.01, kernel=poly;, score=0.917 total time=   0.2s
[CV 2/10] END C=1000, degree=3, gamma=0.01, kernel=poly;, score=0.983 total time=   0.2s
[CV 3/10] END C=1000, degree=3, gamma=0.01, kernel=poly;, score=0.934 total time=   0.2s
[CV 4/10] END C=1000, degree=3, gamma=0.01, kernel=poly;, score=0.926 total time=   0.2s
[CV 5/10] END C=1000, degree=3, gamma=0.01, kernel=poly;, score=0.975 total time=   0.2s
[CV 6/10] END C=1000, degree=3, gamma=0.01, kernel=poly;, score=0.958 total time=   0.2s
[CV 7/10] END C=1000, degree=3, gamma=0.01, kernel=poly;, score=0.967 total time=   0.2s
[CV 8/10] END C=1000, degree=3, gamma=0.01, kernel=poly;, score=0.958 total time=   0.2s
[CV 9/10] END C=1000, degree=3, gamma=0.01, kernel=poly;, score=0.958 total time=   0.2s
[CV 10/10] END C=1000, degree=3, gamma=0.01, kernel=poly;, score=0.942 total time=   0.2s
[CV 1/10] END C=1000, degree=3, gamma=0.01, kernel=linear;, score=0.876 total time=   0.2s
[CV 2/10] END C=10

In [45]:
model_SVC=SVC(C=1, gamma=0.1, kernel='poly', degree=2) #parameters are gridsearched in original implementation.
model_SVC.fit(X_train,Y_train) #CASIA1

SVC(C=1, degree=2, gamma=0.1, kernel='poly')

In [48]:
# Testing acc.
predictions=model_SVC.predict(X_test)

print(accuracy_score(Y_test,predictions))
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))

0.9555125725338491
[[247  18]
 [  5 247]]
              precision    recall  f1-score   support

           0       0.98      0.93      0.96       265
           1       0.93      0.98      0.96       252

    accuracy                           0.96       517
   macro avg       0.96      0.96      0.96       517
weighted avg       0.96      0.96      0.96       517

